<a href="https://colab.research.google.com/github/Xelvise/FineTuning-pretrained-models-with-HuggingFace/blob/main/Finetuning_DistilBERT_for_multi_class_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This project entails finetuning DistilBERT model on a bunch of News articles from BBC. It's a Text classification task having 5 unique classes.

### The dataset can be accessed [here](https://drive.google.com/file/d/1Bveq1oK_Z4g49cIPLNtY3e2EU8qxuZnS/view?usp=sharing)

In [ ]:
!pip install -U transformers datasets accelerate
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import DataCollatorWithPadding

import tensorflow as tf
import pandas as pd
import json
import gc
import torch

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bbc-text.csv')
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [ ]:
df['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [ ]:
df['text_count'] = df['text'].apply(lambda x: len(x.split()))
df['labels'] = df['category'].astype('category').cat.codes
df

,category,text,text_count,labels
0,tech,tv future in the hands of viewers with home th...,737,4
1,business,worldcom boss left books alone former worldc...,300,0
2,sport,tigers wary of farrell gamble leicester say ...,246,3
3,sport,yeading face newcastle in fa cup premiership s...,341,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,260,1
...,...,...,...,...
2220,business,cars pull down us retail figures us retail sal...,293,0
2221,politics,kilroy unveils immigration policy ex-chatshow ...,156,2
2222,entertainment,rem announce new glasgow concert us band rem h...,301,1
2223,politics,how political squabbles snowball it s become c...,858,2


In [ ]:
x = list(df.text)
y = list(df.labels)

# train-test split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
import datasets
from datasets import DatasetDict, Dataset, Features, Value

# Assuming you have your data in the following format
train_data = {'text': xtrain, 'label': ytrain}
test_data = {'text': xtest, 'label': ytest}

# Define the features of your dataset
features = Features({'text': Value('string'), 'label': Value('int32')})

# Create datasets from your data
train_dataset = Dataset.from_dict(train_data, features=features)
test_dataset = Dataset.from_dict(test_data, features=features)

# Create a DatasetDict
data = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1668
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 557
    })
})

In [ ]:
checkpoint = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

# define tokenization method

def tokenize(batch):    # for every data batch (like train, test, validation), this function tokenizes each
    return tokenizer(batch['text'], truncation=True, padding=True, return_tensors='pt')     # batch_size defaults to length of longest sequence in a batch

encoded_data = data.map(tokenize, batched=True)     # batched=True allows for parallel tokenization of all the batches
encoded_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/557 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1668
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 557
    })
})

### Fine-tuning with Pytorch Trainer class

In [ ]:
from transformers import Trainer, TrainingArguments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    # Use GPU if present, else use CPU

# Initialize the classication head adding the expected number of labels
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=5).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# SequenceClassification is mainly for sentiment analysis or text classification tasks
# define hyperparameters for fine-tuning

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/finetuned-multiclass-classifier",
    num_train_epochs=4,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.00001,
    evaluation_strategy='epoch',
    disable_tqdm=False
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
# For every epoch, evaluation is done in which actual and predicted labels is computed

def compute_metrics(pred):
    labels = pred.label_ids     # actual labels
    preds = pred.predictions.argmax(-1)      # predicted label
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy":acc, "f1":f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=encoded_data['train'],
    eval_dataset=encoded_data['test'],
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.107978,0.985637,0.985614
2,No log,0.052465,0.987433,0.987402
3,No log,0.045433,0.989228,0.989216
4,No log,0.049255,0.989228,0.989216


TrainOutput(global_step=420, training_loss=0.2012182144891648, metrics={'train_runtime': 366.125, 'train_samples_per_second': 18.223, 'train_steps_per_second': 1.147, 'total_flos': 883869768990720.0, 'train_loss': 0.2012182144891648, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0492551326751709,
 'eval_accuracy': 0.9892280071813285,
 'eval_f1': 0.9892157909878787,
 'eval_runtime': 10.0519,
 'eval_samples_per_second': 55.413,
 'eval_steps_per_second': 0.895,
 'epoch': 4.0}

In [ ]:
print(data['test']['text'][0])
print(data['test']['label'][0])

drive to  save  festive holidays efforts are being made to  protect  workers  days off on christmas day and new year s day.  support is being sought for a bill which would ensure that large retailers in scotland remain closed on christmas day and 1 january. the usdaw trade union said shop workers should be able to enjoy a break with their families. msp karen whitefield wants to ensure only those whose roles are essential are at work over the festive season. in recent years  more stores have been opening on traditional holidays  with some starting their end-of-year sale on christmas day or new year s day. ms whitefield said:  i have found members of the public to be very supportive when i have been campaigning on the streets.   the early evidence shows quite clearly that the vast majority of people believe that shop workers should be given these two special days to spend with friends and family.  usdaw general secretary john hannett added:  christmas day and new year s day are special d

In [ ]:
print(trainer.predict(encoded_data['test']['text'][0]))

### Model Inference

In [ ]:
model.save_pretrained('/saved_models')
tokenizer.save_pretrained('/saved_models')

model_fine_tuned = DistilBertForSequenceClassification.from_pretrained("/saved_models")

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model=model_fine_tuned, tokenizer=tokenizer)

example = data['test']['text'][0]

classifier(example)

[{'label': 'LABEL_2', 'score': 0.9893737435340881}]